# Intel® System Debugger

The System Debug feature of the Intel® System Debugger is a JTAG-based debug solution supporting in-depth debugging and tracing of Intel® Architecture-based System Software and Embedded Applications. It enables developers to debug and trace Intel® Architecture based platforms system-wide, e.g. UEFI / firmware, System-on-Chip peripheral registers, OS kernel and drivers with full OS awareness.

The Intel® System Debugger - System Debug provides Windows* hosted cross-debug solutions for software developers to debug UEFI BIOS, the Linux* kernel sources and dynamically loaded drivers and kernel modules on Intel Atom® processor and Intel® Core™ based devices.

There are two primary ways to establish connection to target- DAL and OpenIPC. We will cover both these approaches in this demo.

## Using DAL 

<img src="PythonCLI01.PNG" alt="PythonCLI-01" style="width:80%;align: left"/>

The Intel® DFx Abstraction Layer (Intel® DAL) provides a rich source of functionality for debugging target platforms and associated silicon. The Intel DAL abstracts away the target-specific details on how to access the features of interest, such as model-specific registers (MSRs), entering and exiting probe mode, various forms of hardware break detects, and more. The Python* Command Line Interface (Python* CLI) provides access to this functionality in an easy-to-use form while still allowing access to the more advanced features of the Intel DAL

In [ ]:
#Change the directory to Intel's DAL from Intel® System Debugger
#Note that the version of DAL installed on the demo machine should match the version given in this cell

%cd "C:/Intel/DAL_1.1831.340.110"

In [ ]:
#Select the right config file based on the target platform

! ConfigConsole.exe

In [ ]:
#Establish target connection; wait for a while and observe green lights on the CCA

! python itpii/startup.py

In [ ]:
# Call the itpii.baseaccess() command to get an instance of the itpii.ItpII class

import sys, os
dalDir = r"C:/Intel/DAL_1.1831.340.110"
sys.path.insert(0, dalDir)
import itpii
itp = itpii.baseaccess()

In [ ]:
# Check DAL version

itp.cv.version

### Enter/Exit Probe Mode

In [ ]:
# Put all the threads into probe mode

# Output shows a series of messages which describes how each thread entered probe mode and at what address they 
# were executing at the time they were halted

itp.halt()
itp.status()

In [ ]:
# Resume execution of all threads

itp.go()
itp.status()

In [ ]:
# Show current breakall setting

itp.cv.breakall

In [ ]:
# ***********************************************************************************

# Use the control variable called 'breakall' to change the behavior of the Intel DAL in regards to what happens to other threads 
# when a specific thread is told to halt (that is, enter probe mode). 

# If breakall = "off", DAL halts only one specific thread and leaves the other threads running

# If breakall = "global", DAL halts all threads when one specific thread is halted

# Keep any one statement active below, based on the thread halt behavior desired; default value is breakall = "global"

# ***********************************************************************************

# itp.cv.breakall = "off"
itp.cv.breakall = "global"

In [ ]:
# Demonstrate the use of breakall- part 1

# Below statements halt only the first thread, and the other threads are kept running

itp.threads[0].halt()
itp.status()

In [ ]:
# Demonstrate the use of breakall- part 2

# Below statements resume the first thread which was halted in the previous step; the other threads are not affected

itp.threads[0].go()
itp.status()

### View/Modify Registers

In [ ]:
# Create aliases for threads

t0 = itp.threads[0]
t1 = itp.threads[1]
t2 = itp.threads[2]
t3 = itp.threads[3]

In [ ]:
# View desired registers for debug

# Use "Tab" key to get a list of registers using the below template:
# print t0.state.regs.<tab>

print t0.state.regs.rip
print t0.state.regs.rflags

In [ ]:
# Save rflags value

savedRflags = t0.state.regs.rflags.value

In [ ]:
# Update rflags value to all 1s (0xfffffffff)

t0.state.regs.rflags = 0xfffffffff
print "Updated rflags value:\n%s"%t0.state.regs.rflags

print "\n\nExpanded view of rflags:"
t0.state.regs.rflags

In [ ]:
# Restore rflags value

t0.state.regs.rflags = savedRflags
print "Restored flags value is:\n%s"%t0.state.regs.rflags

print "\n\nExpanded view of rflags:"
t0.state.regs.rflags

### Check/Modify Target Status

In [ ]:
# Check target power status

itp.cv.targpower

In [ ]:
# Check processor mode

t0.cv.asmmode

In [ ]:
# List the known TAP devices; the devices primarily hold information about the silicon or portion of silicon it is representing

itp.devicelist

In [ ]:
# Display processor's running state and execution mode

itp.status()

In [ ]:
# Display the cause and address of the recent break event for the selected threads specified by the variable tid

tid = 1

print "Status of recent break event for thread[%d]:"%tid
itp.threads[tid].cv.cause

In [ ]:
# Perform disassembly for the n instructions from the current instruction pointer for the selected threads 
# specified by the variable tid

tid = 0
n   = 10
itp.threads[tid].asm("$", n)

In [ ]:
# Dump memory contents as specifed below:
# tid                --> selected thread
# start_addr         --> first address from where to start dump
# no_of_words        --> number of words to be read in the dump file
# wordsize_in_bytes  --> 1 for 8 bit word
#                        2 for 16 bit word
#                        4 for 32 bit word 
#                        8 for 64 bit word 

tid = 0
start_addr = "0x89E5DE00P"
no_of_words = 25
wordsize_in_bytes = 4

itp.threads[tid].memdump(start_addr, no_of_words, wordsize_in_bytes)

### Handling Breakpoints

In [ ]:
# Set a breakpoint

itp.brnew("0x100P", "io")

In [ ]:
# View breakpoints

itp.brget()

### Usage of Debug Scripts

In [ ]:
# Demonstrates the use of writing a small script to perform a specific debugging task

# Function resetpower() powercycles the target, puts all the cores in probe mode and 
# displays the status of all the halted cores

def resetpower():     
    import os  
    itp.sleep(3)
    itp.resettarget()
    if t0.cv.isrunning:
        itp.halt()
    itp.status()

resetpower()

## Using OpenIPC

In [1]:
! python -V

Python 2.7.15


In [2]:
import ipccli
itp = ipccli.baseaccess()

Connecting to IPC API....
IPC-CLI: 1.1839.957.100, OpenIPC:Main (rev 609620) : 1.1839.3251.200
Initializing IPC API....



In [3]:
itp.cv.version

'IPC-CLI Version 1.1839.957.100'

### Enter/Exit Probe Mode

In [6]:
# Put all the threads into probe mode

# Output shows a series of messages which describes how each thread entered probe mode and at what address they 
# were executing at the time they were halted
itp.halt()
itp.status()

Status for       : GLM_C0_T0
  Processor      : Halted
  Processor mode : Unknown
Status for       : GLM_C2_T0
  Processor      : Halted
  Processor mode : Unknown



In [5]:
# Resume execution of all threads
itp.go()
itp.status()

Status for       : GLM_C0_T0
  Processor      : Running
  Processor mode : Unavailable while Running
Status for       : GLM_C2_T0
  Processor      : Running
  Processor mode : Unavailable while Running



### View/Modify Registers

In [7]:
# Create aliases for threads
t0 = itp.threads[0]
t1 = itp.threads[1]

In [8]:
# View desired registers for debug

# Use "Tab" key to get a list of registers using the below template:
# print t0.state.regs.<tab>

print t0.state.regs.ebx
print t0.state.regs.rflags
print t0.state.regs.eip

[64b] 0x0000000077DAC098
[64b] 0x0000000000000206
[64b] 0x000000007780A461


In [9]:
savedRflags = t0.state.regs.rflags

In [10]:
# Update rflags value to all 1s (0xfffffffff)

t0.state.regs.rflags = 0xfffffffff
print "Updated rflags value:\n%s"%t0.state.regs.rflags

Updated rflags value:
[64b] 0x0000000FFFFFFFFF


In [11]:
# Restore rflags value

t0.state.regs.rflags = savedRflags
print "Restored flags value is:\n%s"%t0.state.regs.rflags

Restored flags value is:
[64b] 0x0000000000000206


In [12]:
# Check MSR (Model Specific Register) values

itp.threads.msr(0x200)

GLM_C0_T0.msr - [64b] 0x00000000FF800001
GLM_C2_T0.msr - [64b] 0x00000000FF800001

### Check/Modify Target Status

In [13]:
# Check target power status

itp.cv.targpower

True

In [15]:
# Check processor mode for the selected threads specified by the variable 'tid'

tid = 1
itp.threads[tid].cv.asmmode

'use32'

In [16]:
# List the known TAP devices; the devices primarily hold information about the silicon or portion of silicon it is representing

itp.devicelist

Indx DID         Alias                                    Type                                    Step Idcode      P/D/ C/T  Enabled
--------------------------------------------------------------------------------------------------------------------------------------------
0    0x00003000  BXTP_CLTAPC0                             BXTP_CLTAPC                             B3   0x00A89013   0/-/ -/-  Yes
1    0x00004000  NORTHPEAK_TAP0                           NORTHPEAK_TAP                           B3   0x00100023   0/-/ -/-  Yes
2    0x00004001  RDU_SE_TAP0                              RDU_SE_TAP                              B3   0x0210002D   0/-/ -/-  Yes
3    0x00004002  CDU_SATAPCIE_TAP0                        CDU_SATAPCIE_TAP                        B3   0x02101613   0/-/ -/-  Yes
4    0x00004003  CDU_SATAPCIE_SCAN_TAP0                   CDU_SATAPCIE_SCAN_TAP                   B3   0x00100915   0/-/ -/-  Yes
5    0x00004004  CDU_SATAPCIE_RETIME_TAP0                 CDU_SATAPCIE_RETIM

In [17]:
# Display processor's running state and execution mode
itp.threads.thread_status()

Status for       : GLM_C0_T0
  Processor      : Halted
  Processor mode : Unknown
Status for       : GLM_C2_T0
  Processor      : Halted
  Processor mode : Unknown


In [19]:
# Display the address of the last known break event for the selected threads specified by the variable tid

tid = 1
print itp.threads[tid].break_address()

0x38:000000007780a461


In [22]:
# Dump memory contents as specifed below:
# tid                --> selected thread
# start_addr         --> first address from where to start dump
# no_of_words        --> number of words to be read in the dump file
# wordsize_in_bytes  --> 1 for 8 bit word
#                        2 for 16 bit word
#                        4 for 32 bit word 
#                        8 for 64 bit word 

tid = 0
start_addr = "0x7780a461P"
no_of_words = 50
wordsize_in_bytes = 4

itp.threads[tid].memdump(start_addr, no_of_words, wordsize_in_bytes)

000000007780a461P: ccccccc3 cccccccc cccccccc 0fcccccc
000000007780a471P: 220fd820 ccccc3d8 cccccccc 9bcccccc
000000007780a481P: 2dd9e3db 000087f7 48e0200f 0002000d
000000007780a491P: e0220f00 e615ae0f c3000087 6acccccc
000000007780a4a1P: b8485000 7780a6b1 00000000 016ae0ff
000000007780a4b1P: b1b84850 007780a6 ff000000 50026ae0
000000007780a4c1P: a6b1b848 00007780 e0ff0000 4850036a
000000007780a4d1P: 80a6b1b8 00000077 6ae0ff00 b8485004
000000007780a4e1P: 7780a6b1 00000000 056ae0ff b1b84850
000000007780a4f1P: 007780a6 ff000000 50066ae0 a6b1b848
000000007780a501P: 00007780 e0ff0000 4850076a 80a6b1b8
000000007780a511P: 00000077 6ae0ff00 b8485008 7780a6b1
000000007780a521P: 00000000 096ae0ff


### Handling Breakpoints

In [ ]:
# Set a breakpoint

In [24]:
%%capture
itp.brnew("0x100P", "io")

In [ ]:
# View breakpoints

In [25]:
itp.brget()

[Breakpoint Name: TCI_BPT_ID#10020000 - Id: 268566528
   Address: 0x0000000000000100P - Data Size: 1 - DBReg: 0 - Enabled: True,
 Breakpoint Name: TCI_BPT_ID#10220000 - Id: 270663680
   Address: 0x0000000000000100P - Data Size: 1 - DBReg: 0 - Enabled: True]

### Basic Bitwise Operations

In [26]:
reg1 = itp.BitData(8, 0xab)
reg2 = itp.BitData(8, 0xcd)
print "reg1 in hex = " +reg1.ToHex()
print "reg2 in hex = " +reg2.ToHex()

# Append reg2 to MSB of reg1

reg3 = reg1.Append(reg2)

# Display reg3 in hex and binary
print "reg3 in hex = " +reg3.ToHex()
print "reg3 in binary = " +reg3.ToBinary()

# Count number of 1s in a byte/word
total_one_bits = reg3.PopCount()
print "total_one_bits in reg3= %d"%total_one_bits


reg1 in hex = 0xAB
reg2 in hex = 0xCD
reg3 in hex = 0xCDAB
reg3 in binary = 0b1100110110101011
total_one_bits in reg3= 10


## Launch System Debugger GUI

In [ ]:
%%capture
! C:\IntelSWTools\system_studio_2018_nda\iss_ide_eclipse-launcher.bat